In [4]:
import os
import cv2
import pandas as pd
import numpy as np
import kagglehub
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers, models, utils

# Загрузка данных
download_path = kagglehub.dataset_download("dmitryyemelyanov/chinese-traffic-signs")

annotations_path = os.path.join(download_path, 'annotations.csv')
images_path = os.path.join(download_path, 'images')

# Чтение аннотаций
annotations = pd.read_csv(annotations_path)

# Список нужных категорий
categories_to_use = [3, 4, 5, 7, 11, 16, 17, 26, 30, 35, 43, 55]

# Фильтрация категорий
annotations_filtered = annotations[annotations['category'].isin(categories_to_use)]

# Словарь для преобразования категорий в индексы
category_mapping = {category: idx for idx, category in enumerate(categories_to_use)}

# Функция для загрузки и обрезки изображений
def load_and_crop_image(row):
    img_path = os.path.join(images_path, row['file_name'])
    image = cv2.imread(img_path)
    if image is None:
        return None
    # Обрезаем изображение по координатам
    x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
    cropped_image = image[y1:y2, x1:x2]
    return cropped_image

# Загрузка изображений
images = []
labels = []

for _, row in annotations_filtered.iterrows():
    img = load_and_crop_image(row)
    if img is not None:
        images.append(cv2.resize(img, (64, 64)))  # Изменяем размер изображений
        labels.append(category_mapping[row['category']])  # Преобразуем категорию в индекс

X = np.array(images)
y = utils.to_categorical(labels, num_classes=len(categories_to_use))  # One-hot кодирование для 12 классов

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
# Построение модели с аугментация
model_with_aug = models.Sequential([
    layers.Input(shape=(64, 64, 3)),

    # Аугментация данных
    layers.RandomTranslation(height_factor=0.02, width_factor=0.02, fill_mode="nearest"),
    layers.RandomRotation(factor=0.01),
    layers.RandomZoom(height_factor=0.05, width_factor=0.05),
    layers.RandomBrightness(factor=(-0.1, 0.1)),

    # Сверточные слои
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    # Полносвязные слои
    layers.Flatten(),
    layers.Dense(128, activation='relu'),

    # Dropout для борьбы с переобучением
    layers.Dropout(0.5),
    
    # Полносвязный слой с 12 выходами (12 классов)
    layers.Dense(len(categories_to_use), activation='softmax')
])

# Компиляция модели
model_with_aug.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
history = model_with_aug.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Оценка на тестовых данных
test_loss_1, test_acc_1 = model_with_aug.evaluate(X_test, y_test)

Epoch 1/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.3145 - loss: 11.0779 - val_accuracy: 0.8672 - val_loss: 0.4709
Epoch 2/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.7915 - loss: 0.6528 - val_accuracy: 0.9520 - val_loss: 0.1684
Epoch 3/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.9151 - loss: 0.2875 - val_accuracy: 0.9852 - val_loss: 0.0788
Epoch 4/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.9507 - loss: 0.1845 - val_accuracy: 0.9852 - val_loss: 0.0569
Epoch 5/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.9391 - loss: 0.1789 - val_accuracy: 0.9815 - val_loss: 0.0524
Epoch 6/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.9588 - loss: 0.1362 - val_accuracy: 0.9926 - val_loss: 0.0253
Epoch 7/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.9808 - loss: 0.0775 - val_accuracy: 1.0000 - val_loss: 0.0085
Epoch 8/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.9661 - loss: 0.1012 - val_accuracy: 0.9871 - 

In [50]:
# Построение модели без аугментация
model_wo_aug = models.Sequential([
    layers.Input(shape=(64, 64, 3)),
    
    # Сверточные слои
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    # Полносвязные слои
    layers.Flatten(),
    layers.Dense(128, activation='relu'),

    # Dropout для борьбы с переобучением
    layers.Dropout(0.5),
    
    # Полносвязный слой с 12 выходами (12 классов)
    layers.Dense(len(categories_to_use), activation='softmax')
])

# Компиляция модели
model_wo_aug.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
history = model_wo_aug.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Оценка на тестовых данных
test_loss_2, test_acc_2 = model_wo_aug.evaluate(X_test, y_test)


Epoch 1/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.3122 - loss: 10.9224 - val_accuracy: 0.7897 - val_loss: 0.6493
Epoch 2/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.7434 - loss: 0.7211 - val_accuracy: 0.9373 - val_loss: 0.1836
Epoch 3/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.8577 - loss: 0.4027 - val_accuracy: 0.9797 - val_loss: 0.0874
Epoch 4/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.8988 - loss: 0.2839 - val_accuracy: 0.9834 - val_loss: 0.0650
Epoch 5/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9361 - loss: 0.1850 - val_accuracy: 0.9797 - val_loss: 0.0684
Epoch 6/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.9478 - loss: 0.1484 - val_accuracy: 0.9889 - val_loss: 0.0404
Epoch 7/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.9560 - loss: 0.1536 - val_accuracy: 0.9908 - val_loss: 0.0224
Epoch 8/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.9597 - loss: 0.1100 - val_accuracy: 0.9889 - 

In [52]:
print("----------------------------------------------------------------")
print()
print(f"Размер тестовой выборки: {len(X_test)} изображений (20%)")
print()
print("----------------------------------------------------------------")
print()
print(f"Модели с аугментацией данных 30 эпох:")
model_with_aug.summary()
print(f"Точность на тестовой выборке: {test_acc_1 * 100:.2f}%")
print(f"Потери на тестовой выборке: {test_loss_1:.4f}")
print()
print("----------------------------------------------------------------")
print()
print(f"Модели без аугментацией данных 10 эпох:")
model_wo_aug.summary()
print(f"Точность на тестовой выборке: {test_acc_2 * 100:.2f}%")
print(f"Потери на тестовой выборке: {test_loss_2:.4f}")
print()
print("----------------------------------------------------------------")


----------------------------------------------------------------

Размер тестовой выборки: 542 изображений (20%)

----------------------------------------------------------------

Модели с аугментацией данных 30 эпох:


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_translation_22           │ (None, 64, 64, 3)      │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_22              │ (None, 64, 64, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_9 (RandomZoom)      │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_brightness_22            │ (None, 64, 64, 3)      │             0 │
│ (RandomBrightness)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_66 (Conv2D)              │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_66 (MaxPooling2D) │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_67 (Conv2D)              │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_67 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_68 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_68 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,054,246 (7.84 MB)

 Trainable params: 684,748 (2.61 MB)

...

 Total params: 2,054,246 (7.84 MB)

 Trainable params: 684,748 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,369,498 (5.22 MB)

Точность на тестовой выборке: 99.26%
Потери на тестовой выборке: 0.0235

----------------------------------------------------------------
